<a href="https://colab.research.google.com/github/jwoonge/ML-projects/blob/master/09/assignment_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification for Multiple Categories using Pytorch
20141261 송제웅  
<hr>  

## 0. Import library
---
import library

In [1]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import matplotlib.pyplot as plt

## 1. DownLoad and Normalize dataset (MNIST)
---  

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,)),
])

data_path = './MNIST'
data_train = datasets.MNIST(root = data_path, train = True, download = True, transform = transform)
data_test = datasets.MNIST(root = data_path, train = False, download = True, transform = transform)


## 2. Design NN Model
---
three fully connected layers with an activation function of sigmoid, LogSoftmax

In [3]:
class classification(nn.Module):
    def __init__(self):
        super(classification, self).__init__()
        self.classifier1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=20*20),
            nn.Sigmoid(),
        )
        self.classifier2 = nn.Sequential(
            nn.Linear(in_features=20*20, out_features=10*10),
            nn.Sigmoid(),
        )
        self.classifier3 = nn.Sequential(
            nn.Linear(in_features=10*10, out_features=10),
            nn.LogSoftmax(dim=1),
        )
        
    def forward(self, inputs):
        x = inputs.view(inputs.size(0), -1)
        x = self.classifier1(x)
        x = self.classifier2(x)
        out = self.classifier3(x)
        return out

## 3. Learning
---

In [103]:
def train(model, data_train, data_train_batch, optimizer, criterion, device='cuda'):
    n_batch = 0
    avg_loss = 0
    avg_acc = 0
    for batch_idx, (x, y) in enumerate(data_train_batch):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model.forward(x)
        loss = criterion(pred, y)
        avg_loss += loss.item()
        avg_acc += accuracy(pred, y)
        n_batch += 1

        loss.backward()
        optimizer.step()

    avg_loss /= n_batch
    avg_acc /= n_batch
    return avg_loss, avg_acc

def test(model, x, y, criterion):
    pred = model.forward(x)
    loss = criterion(pred, y).item()
    acc = accuracy(pred, y)
    return loss, acc

def accuracy(pred, y):
    correct_cnt = 0
    num_sample = len(y)
    for i in range(num_sample):
        pred_class = pred[i].argmax()
        if y[i]==pred_class:
            correct_cnt += 1
    return 100 * correct_cnt / num_sample


In [111]:
def learn(model, data_train, data_test, criterion, optimizer, batch_size, epoch, device='cuda'):
    data_train_batch = torch.utils.data.DataLoader(data_train, batch_size)
    test_x, test_y = data_test.test_data.view((10000,28*28)), data_test.test_labels
    test_x, test_y = torch.tensor(test_x, dtype=torch.float, device=device), test_y.to(device)
    loss_train_s = []; loss_test_s = []; acc_train_s = []; acc_test_s = []
    for i in range(epoch):
        print(i)
        loss_test, acc_test = test(model, test_x, test_y, criterion)
        loss_train, acc_train = train(model, data_train, data_train_batch, optimizer, criterion, device)
        loss_train_s.append(loss_train); loss_test_s.append(loss_test)
        acc_train_s.append(acc_train); acc_test_s.append(acc_test)
    return loss_train_s, loss_test_s, acc_train_s, acc_test_s
        

In [101]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')
learning_rate = 0.01
classifier = classification().to(device)
optimizer = torch.optim.SGD(classifier.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

## 4. Learn with batch size of 32
---

In [ ]:
loss_train_32, loss_test_32, acc_train_32, acc_test_32 = learn(classifier, data_train, data_test, criterion, optimizer, 32, 100)

## 5. Learn with batch size of 64

In [112]:
loss_train_64, loss_test_64, acc_train_64, acc_test_64 = learn(classifier, data_train, data_test, criterion, optimizer, 64, 100)

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


## 6. Learn with batch size of 128

In [113]:
loss_train_128, loss_test_128, acc_train_128, acc_test_128 = learn(classifier, data_train, data_test, criterion, optimizer, 128, 100)

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


---  
# RESULTS
---  
## 1. Plot the training and testing losses with a batch size of 32
---

## 2. Plot the training and testing accuracies with a batch size of 32
---

## 3. Plot the training and testing losses with a batch size of 64
---

## 4. Plot the training and testing accuracies with a batch size of 64
---

## 5. Plot the training and testing losses with a batch size of 128
---

## 6. Plot the training and testing accuracies with a batch size of 128
---

## 7. Print the loss at convergence with different mini-batch sizes
---

## 8. Print the accuracy at convergence with different mini-batch sizes
---